In [1]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, TimeDistributed, Reshape, Lambda
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import pdb

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_pathN = "dataset/no_punctuation/1M/wiki_00"
train_pathP = "dataset/cured_text/1M/wiki_00"
valid_pathN = "dataset/no_punctuation/1M/wiki_01"
valid_pathP = "dataset/cured_text/1M/wiki_01"
test_pathN = "dataset/no_punctuation/1M/wiki_02"
test_pathP = "dataset/cured_text/1M/wiki_02"
models_path = "models"
symbols = ["-", ".", "?", "¿", "!", "¡", ",", ":", "_", ";"]

In [3]:
def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", " <eos>").split()

In [4]:
read_words(train_pathN)

['Andorra',
 '<eos>',
 'Andorra',
 'oficialmente',
 'Principado',
 'de',
 'Andorra',
 '(',
 ')',
 'es',
 'un',
 'pequeño',
 'país',
 'soberano',
 'del',
 'suroeste',
 'de',
 'Europa',
 'Constituido',
 'en',
 'Estado',
 'independiente',
 'de',
 'derecho',
 'democrático',
 'y',
 'social',
 'cuya',
 'forma',
 'de',
 'gobierno',
 'es',
 'el',
 'coprincipado',
 'parlamentario',
 'Su',
 'territorio',
 'está',
 'organizado',
 'en',
 'siete',
 'parroquias',
 'con',
 'una',
 'población',
 'total',
 'de',
 '<num>',
 '<num>',
 'habitantes',
 '(',
 '<num>',
 ')',
 'Su',
 'capital',
 'es',
 'Andorra',
 'la',
 'Vieja',
 '<eos>',
 'Tiene',
 '<num>',
 'km²',
 'de',
 'extensión',
 'territorial',
 'y',
 'está',
 'situado',
 'en',
 'los',
 'Pirineos',
 'entre',
 'España',
 'y',
 'Francia',
 'con',
 'una',
 'altitud',
 'media',
 'de',
 '<num>',
 'msnm',
 'Limita',
 'por',
 'el',
 'sur',
 'con',
 'España',
 '—',
 'con',
 'las',
 'comarcas',
 'catalanas',
 'de',
 'Cerdaña',
 'Alto',
 'Urgel',
 'y',
 'Pallar

In [5]:
def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)        
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))        

    words, _ = list(zip(*count_pairs))     
    word_to_id = dict(zip(words, range(len(words))))
    
    return word_to_id

In [6]:
def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

In [7]:
def load_data():
    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_pathP)
    train_dataN = file_to_word_ids(train_pathN, word_to_id)
    train_dataP = file_to_word_ids(train_pathP, word_to_id)
    valid_dataN = file_to_word_ids(valid_pathN, word_to_id)
    valid_dataP = file_to_word_ids(valid_pathP, word_to_id)
    test_dataN = file_to_word_ids(test_pathN, word_to_id)
    test_dataP = file_to_word_ids(test_pathP, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    #print(train_data[:5])
    #print(word_to_id)
    #print(vocabulary)
    #print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_dataN, train_dataP, valid_dataN, valid_dataP, test_dataN, test_dataP, vocabulary, reversed_dictionary

train_dataN, train_dataP, valid_dataN, valid_dataP, test_dataN, test_dataP, vocabulary, reversed_dictionary = load_data()

In [8]:
print(len(train_dataP))
print(" ".join([reversed_dictionary[x] for x in train_dataN[100:110]]))
print(" ".join([reversed_dictionary[x] for x in train_dataP[100:110]]))

188953
Sobirá — y por el norte con Francia — con
con España — con las comarcas catalanas de Cerdaña ,


In [34]:
class KerasBatchGenerator(object):

    def __init__(self, dataN, dataP, num_steps, batch_size, vocabulary, skip_step=5):
        self.dataN = dataN
        self.dataP = dataP
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset back to zero
        self.cnt1 = 0
        self.cnt2 = 0
        # skip_step is the number of words which will be skipped before the next batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.cnt1 + self.num_steps >= len(self.dataN):
                    # reset the index back to the start of the data set
                    self.cnt1 = 0
                    self.cnt2 = 0
                x[i, :] = self.dataN[self.cnt1 : self.cnt1 + self.num_steps]
                temp_y = self.dataP[self.cnt2 : self.cnt2 + self.num_steps]
                #print()
                #print(" ".join([reversed_dictionary[x] for x in self.dataN[self.cnt1 : self.cnt1 + self.num_steps]]))
                #print(" ".join([reversed_dictionary[x] for x in self.dataP[self.cnt2 : self.cnt2 + self.num_steps]]))
                
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                
                self.cnt1 += self.skip_step
                for i in range(self.skip_step):
                    while reversed_dictionary[self.dataP[self.cnt2]] in symbols:                        
                        self.cnt2 += 1
                    self.cnt2 += 1
                
            yield x, y

In [35]:
num_steps = 30
batch_size = 20

train_data_generator = KerasBatchGenerator(train_dataN, train_dataP, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_dataN, valid_dataP, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
hidden_size = 500
use_dropout=True

In [36]:
model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 30, 500)           10771500  
_________________________________________________________________
lstm_11 (LSTM)               (None, 30, 500)           2002000   
_________________________________________________________________
lstm_12 (LSTM)               (None, 30, 500)           2002000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 30, 500)           0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 21543)         10793043  
_________________________________________________________________
activation_6 (Activation)    (None, 30, 21543)         0         
Total params: 25,568,543
Trainable params: 25,568,543
Non-trainable params: 0
________________________________________________________________

In [37]:
#1 to train, 2 to test
run_opt = 1
checkpointer = ModelCheckpoint(filepath = models_path + "/model-{epoch:02d}.hdf5", verbose=1)
num_epochs = 50

In [38]:
if run_opt == 1:
    model.fit_generator(train_data_generator.generate(), len(train_dataP)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_dataP)//(batch_size*num_steps), callbacks=[checkpointer])
    # model.fit_generator(train_data_generator.generate(), 2000, num_epochs,
    #                     validation_data=valid_data_generator.generate(),
    #                     validation_steps=10)
    model.save(models_path + "/final_model.hdf5")

Epoch 1/50

Andorra <eos> Andorra oficialmente Principado de Andorra ( ) es un pequeño país soberano del suroeste de Europa Constituido en Estado independiente de derecho democrático y social cuya forma de
Andorra <eos> Andorra , oficialmente Principado de Andorra ( ) , es un pequeño país soberano del suroeste de Europa . Constituido en Estado independiente , de derecho , democrático

gobierno es el coprincipado parlamentario Su territorio está organizado en siete parroquias con una población total de <num> <num> habitantes ( <num> ) Su capital es Andorra la Vieja <eos>
gobierno es el coprincipado parlamentario . Su territorio está organizado en siete parroquias , con una población total de <num> <num> habitantes ( <num> ) . Su capital es Andorra

Tiene <num> km² de extensión territorial y está situado en los Pirineos entre España y Francia con una altitud media de <num> msnm Limita por el sur con España —
Tiene <num> km² de extensión territorial y está situado en los Pirineos , entre 


" en Lérida que fija los límites del poder de cada señor representa la fundación del Principado de Andorra y define las obligaciones de los andorranos en materia de diezmos
" en Lérida , que fija los límites del poder de cada señor , representa la fundación del Principado de Andorra y define las obligaciones de los andorranos en materia

y asuntos militares por lo que se formó un " condominio feudal " aunque esto no le preocupa a la Corona de Aragón ya que el condado de Foix era
y asuntos militares , por lo que se formó un " condominio feudal " , aunque esto no le preocupa a la Corona de Aragón , ya que el condado

vasallo suyo y Andorra estaba dentro de las fronteras de la corona Diez años después como consecuencia de la construcción de una iglesia fortificada en san Vicente de Enclar por
vasallo suyo y Andorra estaba dentro de las fronteras de la corona . Diez años después , como consecuencia de la construcción de una iglesia fortificada en san Vicente de

parte del conde de Foix de


la presencia de gendarmes franceses comandados nuevamente por el coronel René Baulard hasta <num> año en que Francia se rinde al ejército alemán La Guerra Civil Española y la consiguiente
la presencia de gendarmes franceses , comandados nuevamente por el coronel René Baulard hasta <num> año en que Francia se rinde al ejército alemán . La Guerra Civil Española y

victoria del bando franquista además del estallido de la II Guerra Mundial y la invasión de Francia por parte de los alemanes de Hitler dejó al país en una difícil
victoria del bando franquista , además del estallido de la II Guerra Mundial y la invasión de Francia por parte de los alemanes de Hitler , dejó al país en

tesitura amparada por el Gobierno de Vichy dirigido por el Mariscal Petain tuvo que simpatizar forzosamente con los regímenes de Hitler y de Francisco Franco para mantener su neutralidad aprovechando
tesitura , amparada por el Gobierno de Vichy dirigido por el Mariscal Petain , tuvo que simpatizar forzosamente c


Juclar cuya superficie es la más extensa de todos los lagos del Principado con <num> hectáreas y que durante la época estival y por la consecuente falta de agua puede
Juclar , cuya superficie es la más extensa de todos los lagos del Principado con <num> hectáreas y que , durante la época estival y por la consecuente falta de

avistarse como tres lagos diferentes aunque en realidad es el mismo el lago de l ' Illa con <num> hectáreas el estanque de Engolasters con <num> hectáreas y los tres
avistarse como tres lagos diferentes , aunque en realidad es el mismo , el lago de l ' Illa con <num> hectáreas , el estanque de Engolasters con <num> hectáreas

lagos de Tristaina <eos> La población de Andorra en <num> era de <num> habitantes En <num> era de <num> habitantes de los cuales <num> eran andorranos <num> españoles <num> portugueses
lagos de Tristaina . <eos> La población de Andorra en <num> era de <num> habitantes . En <num> era de <num> habitantes , de los cuales <num> eran andorranos ,

primera serie <eos> Tradicionalmente Andorra emitía monedas denominadas « Diners » que no tenían curso legal sino que eran con fines de coleccionismo Estas monedas podrán seguir siendo emitidas <eos>
primera serie . <eos> Tradicionalmente Andorra emitía monedas denominadas « Diners » que no tenían curso legal sino que eran con fines de coleccionismo . Estas monedas podrán seguir siendo

La supervisión de los bancos andorranos y demás entidades financieras corresponde al Instituto Nacional Andorrano de Finanzas creado en <num> <eos> Andorra se comunica con el exterior a través de
La supervisión de los bancos andorranos y demás entidades financieras corresponde al Instituto Nacional Andorrano de Finanzas , creado en <num> <eos> Andorra se comunica con el exterior a través

las vías terrestres No dispone de tren ( aunque existe un proyecto de línea ferroviaria Barcelona Andorra conectando la Seo de Urgel con Puigcerdá además de la estación Andorra l
las vías terrestres . No dispone de tre

...

 19/314 [>.............................] - ETA: 35:07 - loss: 8.6399 - categorical_accuracy: 0.0686
Agroalimentario Canadá ( AAFC ) Austrade y NZTE que representan cuatro de los países más importantes a nivel de exportación de productos agrícolas La Federación de Asociaciones de Comercio Internacional
Agroalimentario Canadá ( AAFC ) , Austrade y NZTE , que representan cuatro de los países más importantes a nivel de exportación de productos agrícolas . La Federación de Asociaciones

publica estudios de FAS y AAFC así como de otros organismos no gubernamentales en su página web globaltrade net <eos> La filosofía de la agricultura es a grandes rasgos una
publica estudios de FAS y AAFC , así como de otros organismos no gubernamentales , en su página web globaltrade . net . <eos> La filosofía de la agricultura es

disciplina dedicada a la crítica sistemática de los marcos filosóficos ( o cosmovisiones éticas ) que son la base para las decisiones relativas a la agricultura <eos> Muchos de es

 21/314 [=>............................] - ETA: 34:29 - loss: 8.5783 - categorical_accuracy: 0.0674
para controlar dispositivos ( como impresoras ) que usaban ASCII Por ejemplo el carácter <num> representa la función " nueva línea " ( line feed ) que hace que una
para controlar dispositivos ( como impresoras ) que usaban ASCII . Por ejemplo , el carácter <num> representa la función " nueva línea " ( line feed ) , que

impresora avance el papel y el carácter <num> representa la tecla " escape " que a menudo se encuentra en la esquina superior izquierda de los teclados comunes <eos> El
impresora avance el papel , y el carácter <num> representa la tecla " escape " que a menudo se encuentra en la esquina superior izquierda de los teclados comunes .

código <num> ( los siete bits a uno ) otro carácter especial equivale a " suprimir " ( " delete " ) Aunque esta función se asemeja a otros caracteres
código <num> ( los siete bits a uno ) , otro carácter especial , equivale a " suprimir " ( " d

 23/314 [=>............................] - ETA: 34:01 - loss: 8.5458 - categorical_accuracy: 0.0677
y UCS La popular codificación UTF <num> recomienda el uso de uno a cuatro valores de <num> bits para cada apuntador donde los primeros <num> valores apuntan a los mismos
y UCS . La popular codificación UTF - <num> recomienda el uso de uno a cuatro valores de <num> bits para cada apuntador , donde los primeros <num> valores apuntan

caracteres que ASCII Otras codificaciones de caracteres como UTF <num> se parece a ASCII en cómo representan los primeros <num> caracteres de Unicode pero tienden a usar <num> a <num>
caracteres que ASCII . Otras codificaciones de caracteres como UTF - <num> se parece a ASCII en cómo representan los primeros <num> caracteres de Unicode , pero tienden a usar

bits por carácter así que requieren de una conversión adecuada para que haya compatibilidad entre ambos códigos de carácter <eos> La palabra " ASCIIbético " ( o más habitualmente la
bits por carácter , así


aumento de población con sus correspondientes necesidades edificatorias ( especialmente de viviendas ) sino también importantes movimientos de poblaciones desde las zonas rurales a las urbanas y terminada la Guerra
aumento de población , con sus correspondientes necesidades edificatorias ( especialmente de viviendas ) , sino también importantes movimientos de poblaciones , desde las zonas rurales a las urbanas y

Fría desde los países pobres a los países ricos Los movimientos migratorios han supuesto no sólo un aumento de la demanda de nuevos edificios en las zonas urbanas sino también
Fría , desde los países pobres a los países ricos . Los movimientos migratorios han supuesto , no sólo un aumento de la demanda de nuevos edificios en las zonas


KeyboardInterrupt: 

In [ ]:
if run_opt == 2:
    model = load_model(models_path + "/final_model.hdf5")    
    example_training_generator = KerasBatchGenerator(train_dataN, train_dataP, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Training data:")    
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_training_generator.generate())
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps-1, :])
        true_print_out += reversed_dictionary[train_dataP[num_steps + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)
    # test data set    
    example_test_generator = KerasBatchGenerator(test_dataN, test_dataP, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Test data:")    
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_test_generator.generate())
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps - 1, :])
        true_print_out += reversed_dictionary[test_dataP[num_steps + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)